# Brain Tumor Classification with VGG16

This notebook demonstrates an end-to-end workflow for brain tumor classification using VGG16 transfer learning models. It includes data preprocessing, multiple model variants, training, evaluation, and comprehensive result generation.

## Overview

1. Mount Google Drive and set up the environment
2. Update the repository
3. Install dependencies
4. Set up paths to the dataset and results directories
5. Load and explore the dataset
6. Preprocess the data with augmentation
7. Build multiple VGG16 model variants
8. Train and validate all models
9. Evaluate performance with comprehensive metrics
10. Generate visualizations and save all required outputs

## 1. Mount Google Drive and Setup Environment

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify TensorFlow and GPU availability
import tensorflow as tf
import platform

print('TensorFlow version:', tf.__version__)
print('Python version:', platform.python_version())
print('GPUs available:', tf.config.list_physical_devices('GPU'))

# Set seed for reproducibility
import numpy as np
import random
import os

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## 2. Update Repository and Install Dependencies

In [ ]:
# Navigate to the main directory and install required packages
%cd /content/drive/MyDrive/BrainTumor

# Install required packages
!pip install -q opencv-python
!pip install -q scikit-learn
!pip install -q matplotlib
!pip install -q seaborn

print("✅ Dependencies installed successfully")

## 3. Import Required Libraries

In [ ]:
# Import all required libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import json
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# Import custom modules - assuming code is in BrainTumor directory
import sys
sys.path.append('/content/drive/MyDrive/BrainTumor')

# If you have the src folder structure in BrainTumor, uncomment these:
# from src.common.dataset_utils import create_datasets
# from src.common.preprocessing import get_augmentation_pipeline, verify_dataset, split_and_copy
# from src.common.gradcam import generate_gradcam

print("✅ All libraries imported successfully")

## 3.1. Define Utility Functions

Since we're working directly from the BrainTumor directory, let's define the utility functions here:

In [ ]:
# Essential utility functions for data processing
from sklearn.model_selection import train_test_split
from collections import Counter
import shutil

def load_image_paths(data_dir, allowed_classes=None):
    """Scan a directory and collect image paths and labels."""
    data_dir = Path(data_dir)
    all_dirs = [p for p in data_dir.iterdir() if p.is_dir()]
    
    if allowed_classes:
        classes = [c for c in allowed_classes if (data_dir / c).is_dir()]
    else:
        classes = sorted([p.name for p in all_dirs])

    filepaths, labels = [], []
    for cls in classes:
        cls_dir = data_dir / cls
        for img_path in cls_dir.glob('*'):
            if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                filepaths.append(str(img_path))
                labels.append(cls)
    
    print(f"✅ Found {len(filepaths)} images in {classes}")
    return filepaths, labels, classes

def create_splits(filepaths, labels, test_size=0.15, val_size=0.15, seed=42):
    """Create train/val/test splits."""
    t_files, te_files, t_labels, te_labels = train_test_split(
        filepaths, labels, test_size=test_size, stratify=labels, random_state=seed
    )
    tr_files, va_files, tr_labels, va_labels = train_test_split(
        t_files, t_labels, test_size=val_size/(1-test_size), stratify=t_labels, random_state=seed
    )
    
    print("Split sizes:", {'train': len(tr_files), 'val': len(va_files), 'test': len(te_files)})
    print("Train label counts:", dict(Counter(tr_labels)))
    print("Val label counts:", dict(Counter(va_labels)))
    print("Test label counts:", dict(Counter(te_labels)))
    
    return (tr_files, tr_labels), (va_files, va_labels), (te_files, te_labels)

def copy_files_to_split_dir(files, labels, output_dir, split_name):
    """Copy files to train/val/test directories."""
    split_dir = Path(output_dir) / split_name
    
    for cls in set(labels):
        cls_dir = split_dir / cls
        cls_dir.mkdir(parents=True, exist_ok=True)
    
    for file_path, label in zip(files, labels):
        src = Path(file_path)
        dst = split_dir / label / src.name
        shutil.copy2(src, dst)

def split_and_copy(raw_dir, processed_dir, class_names):
    """Split raw data into train/val/test and copy to processed directory."""
    filepaths, labels, classes = load_image_paths(raw_dir, class_names)
    
    if len(filepaths) == 0:
        print("⚠️ No images found!")
        return 0
    
    train_data, val_data, test_data = create_splits(filepaths, labels)
    
    # Copy files to respective directories
    copy_files_to_split_dir(train_data[0], train_data[1], processed_dir, "train")
    copy_files_to_split_dir(val_data[0], val_data[1], processed_dir, "val")
    copy_files_to_split_dir(test_data[0], test_data[1], processed_dir, "test")
    
    return len(filepaths)

def verify_dataset(data_dir):
    """Verify if processed dataset exists with correct structure."""
    data_path = Path(data_dir)
    
    if not data_path.exists():
        return False, False
    
    required_dirs = ["train", "val", "test"]
    required_classes = ["yes", "no"]
    
    for split in required_dirs:
        split_dir = data_path / split
        if not split_dir.exists():
            return False, False
        
        for cls in required_classes:
            cls_dir = split_dir / cls
            if not cls_dir.exists():
                return False, False
    
    return True, True

def get_augmentation_pipeline():
    """Get data augmentation pipeline."""
    return tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ])

def load_and_preprocess_image(path, label, img_size):
    """Load and preprocess a single image with error handling."""
    try:
        # Read and decode image
        image = tf.io.read_file(path)
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
        
        # Ensure the image has a defined shape
        image = tf.ensure_shape(image, [None, None, 3])
        
        # Resize image
        image = tf.image.resize(image, img_size)
        
        # Normalize to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        
        return image, label
    except Exception as e:
        # If image loading fails, return a black image
        print(f"Warning: Could not load image {path}, using placeholder")
        black_image = tf.zeros((*img_size, 3), dtype=tf.float32)
        return black_image, label

def create_tf_dataset(file_paths, labels, class_names, batch_size, img_size, is_training=False, augment_fn=None):
    """Create TensorFlow dataset from file paths and labels."""
    # Convert string labels to integers
    label_to_int = {name: i for i, name in enumerate(class_names)}
    int_labels = [label_to_int[label] for label in labels]
    
    # Create dataset
    ds = tf.data.Dataset.from_tensor_slices((file_paths, int_labels))
    
    # Map the load and preprocess function
    ds = ds.map(
        lambda path, label: load_and_preprocess_image(path, label, img_size),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Filter out any None values (corrupted images)
    ds = ds.filter(lambda image, label: tf.reduce_all(tf.math.is_finite(image)))
    
    if is_training and augment_fn is not None:
        ds = ds.map(lambda x, y: (augment_fn(x), y), num_parallel_calls=tf.data.AUTOTUNE)
    
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    
    return ds

def create_datasets(data_dir, batch_size, img_size=(224, 224), augment_fn=None):
    """Create train/val/test datasets from processed directory."""
    data_path = Path(data_dir)
    
    # Get class names
    train_dir = data_path / "train"
    class_names = sorted([d.name for d in train_dir.iterdir() if d.is_dir()])
    
    datasets = {}
    
    for split in ["train", "val", "test"]:
        split_dir = data_path / split
        file_paths = []
        labels = []
        
        for cls in class_names:
            cls_dir = split_dir / cls
            for img_path in cls_dir.glob("*"):
                if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                    file_paths.append(str(img_path))
                    labels.append(cls)
        
        print(f"Found {len(file_paths)} images in {split} set")
        
        is_training = (split == "train")
        datasets[split] = create_tf_dataset(
            file_paths, labels, class_names, batch_size, img_size, is_training, augment_fn
        )
    
    return datasets["train"], datasets["val"], datasets["test"], class_names

print("✅ Utility functions defined successfully")

In [ ]:
# Simple Grad-CAM implementation
def generate_gradcam(model, dataset, save_dir, class_names, num_images=3, max_samples=2):
    """Generate simple Grad-CAM visualizations."""
    os.makedirs(save_dir, exist_ok=True)
    
    print(f"Generating Grad-CAM visualizations (limited to {num_images} images)...")
    
    # Get a batch of images
    for batch_num, (images, labels) in enumerate(dataset.take(max_samples)):
        if batch_num >= max_samples:
            break
            
        for i in range(min(num_images, len(images))):
            try:
                img = images[i:i+1]
                
                # Find the last convolutional layer
                last_conv_layer = None
                for layer in reversed(model.layers):
                    if len(layer.output_shape) == 4:  # Conv layer has 4D output
                        last_conv_layer = layer
                        break
                
                if last_conv_layer is None:
                    print("No convolutional layer found")
                    continue
                
                # Create a model that maps the input image to the activations of the last conv layer
                grad_model = tf.keras.models.Model(
                    inputs=[model.inputs],
                    outputs=[last_conv_layer.output, model.output]
                )
                
                # Compute the gradient of the predicted class for our input image
                with tf.GradientTape() as tape:
                    conv_outputs, predictions = grad_model(img)
                    loss = predictions[0, 0]  # For binary classification
                
                # Extract gradients
                grads = tape.gradient(loss, conv_outputs)
                
                # Pool the gradients over all the axes leaving out the channel dimension
                pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
                
                # Weight the channels by the corresponding gradients
                conv_outputs = conv_outputs[0]
                heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
                heatmap = tf.squeeze(heatmap)
                
                # Normalize the heatmap
                heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
                
                # Create the visualization
                img_display = images[i].numpy()
                
                plt.figure(figsize=(12, 4))
                
                plt.subplot(1, 3, 1)
                plt.imshow(img_display)
                plt.title('Original Image')
                plt.axis('off')
                
                plt.subplot(1, 3, 2)
                plt.imshow(heatmap, cmap='jet')
                plt.title('Grad-CAM Heatmap')
                plt.axis('off')
                
                plt.subplot(1, 3, 3)
                # Resize heatmap to image size
                heatmap_resized = tf.image.resize(heatmap[..., tf.newaxis], img_display.shape[:2])
                heatmap_resized = tf.squeeze(heatmap_resized)
                
                plt.imshow(img_display)
                plt.imshow(heatmap_resized, cmap='jet', alpha=0.4)
                plt.title('Overlay')
                plt.axis('off')
                
                # Save the visualization
                filename = f"gradcam_batch{batch_num}_img{i}.png"
                plt.savefig(os.path.join(save_dir, filename), dpi=150, bbox_inches='tight')
                plt.close()
                
            except Exception as e:
                print(f"Error generating Grad-CAM for image {i}: {str(e)}")
                continue
    
    print(f"✅ Grad-CAM visualizations saved to {save_dir}")

print("✅ Grad-CAM function defined successfully")

## 4. Setup Paths and Configuration

In [ ]:
# Configuration - corrected paths for actual folder structure
BASE_DIR = "/content/drive/MyDrive/BrainTumor"
RAW_DATA_DIR = os.path.join(BASE_DIR, "data", "archive")  # Raw images are in BrainTumor/data/archive/
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "data", "processed")
RESULTS_DIR = os.path.join(BASE_DIR, "Result", "vgg16")

# Model configuration
INPUT_SHAPE = (224, 224, 3)  # VGG16 optimal input size
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001

# Create necessary directories
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "gradcam"), exist_ok=True)

print(f"✅ Paths configured:")
print(f"   Base directory: {BASE_DIR}")
print(f"   Raw data: {RAW_DATA_DIR}")
print(f"   Processed data: {PROCESSED_DATA_DIR}")
print(f"   Results: {RESULTS_DIR}")
print(f"   Input shape: {INPUT_SHAPE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS}")

# Verify the raw data structure exists
yes_dir = os.path.join(RAW_DATA_DIR, "yes")
no_dir = os.path.join(RAW_DATA_DIR, "no")

print(f"\n🔍 Checking raw data structure:")
print(f"   Raw data directory exists: {os.path.exists(RAW_DATA_DIR)}")
print(f"   Yes folder exists: {os.path.exists(yes_dir)}")
print(f"   No folder exists: {os.path.exists(no_dir)}")

if os.path.exists(yes_dir):
    yes_count = len([f for f in os.listdir(yes_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"   Images in yes folder: {yes_count}")

if os.path.exists(no_dir):
    no_count = len([f for f in os.listdir(no_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"   Images in no folder: {no_count}")

## 5. Data Preprocessing

Check if preprocessing is needed and perform data preprocessing if required.

In [ ]:
# Check if processed data exists
train_dir = os.path.join(PROCESSED_DATA_DIR, "train")
val_dir = os.path.join(PROCESSED_DATA_DIR, "val")
test_dir = os.path.join(PROCESSED_DATA_DIR, "test")

processed_exists, class_folders_valid = verify_dataset(PROCESSED_DATA_DIR)

if processed_exists and class_folders_valid:
    print("✅ Processed data already exists with valid class folders. Skipping preprocessing.")
else:
    print("🔄 Starting preprocessing...")
    print(f"Reading raw images from: {RAW_DATA_DIR}")
    print(f"Saving processed images to: {PROCESSED_DATA_DIR}")

    # Check if raw data exists
    yes_dir = os.path.join(RAW_DATA_DIR, "yes")
    no_dir = os.path.join(RAW_DATA_DIR, "no")
    
    if os.path.exists(RAW_DATA_DIR) and os.path.exists(yes_dir) and os.path.exists(no_dir):
        total_files = split_and_copy(RAW_DATA_DIR, PROCESSED_DATA_DIR, ["yes", "no"])
        print(f"✅ Preprocessing completed successfully! Processed {total_files} images.")
    else:
        print("⚠️  Could not find expected yes/no folders in the raw data directory.")
        print("Please make sure your Google Drive contains the correct folder structure.")

## 6. Load and Explore Dataset

In [ ]:
# Create datasets with data augmentation for VGG16 (224x224 input)
augment = get_augmentation_pipeline()
train_ds, val_ds, test_ds, class_names = create_datasets(
    PROCESSED_DATA_DIR, 
    BATCH_SIZE, 
    img_size=(224, 224),  # VGG16 optimal size
    augment_fn=augment
)

print(f"✅ Dataset loaded successfully!")
print(f"   Class names: {class_names}")
print(f"   Input shape: {INPUT_SHAPE}")

# Display sample images
plt.figure(figsize=(12, 8))
for images, labels in train_ds.take(1):
    for i in range(min(9, len(images))):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Class: {class_names[int(labels[i])]}")
        plt.axis('off')
plt.suptitle("Sample Images from Training Dataset", fontsize=16)
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "sample_images.png"), dpi=300, bbox_inches='tight')
plt.show()

# Dataset statistics
train_samples = sum(1 for _ in train_ds.unbatch())
val_samples = sum(1 for _ in val_ds.unbatch())
test_samples = sum(1 for _ in test_ds.unbatch())

print(f"\n📊 Dataset Statistics:")
print(f"   Training samples: {train_samples}")
print(f"   Validation samples: {val_samples}")
print(f"   Test samples: {test_samples}")
print(f"   Total samples: {train_samples + val_samples + test_samples}")

## 7. Build VGG16 Model Variants

Create multiple VGG16 model variants for comparison:

In [ ]:
def build_vgg16_model(input_shape=INPUT_SHAPE):
    """
    Optimized VGG16 transfer learning model for brain tumor classification
    Uses fine-tuning approach for best performance
    """
    # Load pre-trained VGG16 model
    base_model = VGG16(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape
    )
    
    # Fine-tuning: Freeze early layers, unfreeze last block for better feature learning
    base_model.trainable = True
    for layer in base_model.layers[:-4]:
        layer.trainable = False
    
    # Build the complete model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    # Compile with lower learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build the VGG16 model
print("🏗️ Building VGG16 model...")
vgg16_model = build_vgg16_model()

print(f"\n{'='*60}")
print(f"📋 VGG16 Model Summary")
print(f"{'='*60}")
vgg16_model.summary()

# Save model summary
with open(os.path.join(RESULTS_DIR, "VGG16_model_summary.txt"), "w") as f:
    vgg16_model.summary(print_fn=lambda x: f.write(x + "\n"))

print(f"\n✅ VGG16 model created successfully!")
print(f"   Total parameters: {vgg16_model.count_params():,}")
print(f"   Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in vgg16_model.trainable_weights]):,}")

## 8. Model Training and Validation

Train the VGG16 model and track its performance:

In [ ]:
# Training function
def train_vgg16_model(model, epochs=EPOCHS):
    """Train the VGG16 model with callbacks and return history"""
    
    # Create model directory
    model_dir = os.path.join(RESULTS_DIR, "VGG16")
    os.makedirs(model_dir, exist_ok=True)
    
    # Callbacks for training optimization
    callbacks = [
        ModelCheckpoint(
            filepath=os.path.join(model_dir, "VGG16_best.h5"),
            monitor="val_accuracy",
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    print(f"\n🚀 Training VGG16 model...")
    print(f"   Epochs: {epochs}")
    print(f"   Model directory: {model_dir}")
    print(f"   Callbacks: ModelCheckpoint, EarlyStopping, ReduceLROnPlateau")
    
    # Train the model
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    print(f"✅ VGG16 training completed!")
    print(f"   Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
    print(f"   Total epochs trained: {len(history.history['accuracy'])}")
    
    return history

# Train the VGG16 model
print("="*70)
print("🎯 Starting VGG16 Model Training")
print("="*70)

vgg16_history = train_vgg16_model(vgg16_model)

print(f"\n🎉 VGG16 model training completed successfully!")
print(f"   Final training accuracy: {vgg16_history.history['accuracy'][-1]:.4f}")
print(f"   Final validation accuracy: {vgg16_history.history['val_accuracy'][-1]:.4f}")

## 9. Generate Training Plots

Create and save training history plots for the VGG16 model:

In [ ]:
# Create comprehensive training plots for VGG16
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('VGG16 Model Training Analysis', fontsize=16, fontweight='bold')

# Training and validation accuracy
ax1.plot(vgg16_history.history['accuracy'], label='Training Accuracy', color='blue', linewidth=2)
ax1.plot(vgg16_history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
ax1.set_title('Model Accuracy', fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Training and validation loss
ax2.plot(vgg16_history.history['loss'], label='Training Loss', color='blue', linewidth=2)
ax2.plot(vgg16_history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
ax2.set_title('Model Loss', fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Learning rate (if available)
if 'lr' in vgg16_history.history:
    ax3.plot(vgg16_history.history['lr'], color='green', linewidth=2)
    ax3.set_title('Learning Rate Schedule', fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_yscale('log')
    ax3.grid(True, alpha=0.3)
else:
    # Show accuracy difference instead
    train_acc = vgg16_history.history['accuracy']
    val_acc = vgg16_history.history['val_accuracy']
    acc_diff = [t - v for t, v in zip(train_acc, val_acc)]
    ax3.plot(acc_diff, color='purple', linewidth=2)
    ax3.set_title('Training-Validation Accuracy Gap', fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Accuracy Difference')
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    ax3.grid(True, alpha=0.3)

# Epoch-wise improvement
epochs = range(1, len(vgg16_history.history['val_accuracy']) + 1)
val_acc_improvement = [0] + [vgg16_history.history['val_accuracy'][i] - vgg16_history.history['val_accuracy'][i-1] 
                             for i in range(1, len(vgg16_history.history['val_accuracy']))]
ax4.bar(epochs, val_acc_improvement, color='orange', alpha=0.7)
ax4.set_title('Validation Accuracy Improvement per Epoch', fontweight='bold')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Accuracy Improvement')
ax4.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "training_analysis.png"), dpi=300, bbox_inches='tight')
plt.show()

# Create the main training plot (for comparison with friend's model)
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(vgg16_history.history['accuracy'], label='Training', linewidth=2, color='blue')
plt.plot(vgg16_history.history['val_accuracy'], label='Validation', linewidth=2, color='red')
plt.title('VGG16 Model Accuracy', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(vgg16_history.history['loss'], label='Training', linewidth=2, color='blue')
plt.plot(vgg16_history.history['val_loss'], label='Validation', linewidth=2, color='red')
plt.title('VGG16 Model Loss', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.suptitle('VGG16 Brain Tumor Classification - Training History', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "training_plot.png"), dpi=300, bbox_inches='tight')
plt.show()

# Print training summary
print("📊 Training Summary:")
print(f"   Final Training Accuracy: {vgg16_history.history['accuracy'][-1]:.4f}")
print(f"   Final Validation Accuracy: {vgg16_history.history['val_accuracy'][-1]:.4f}")
print(f"   Best Validation Accuracy: {max(vgg16_history.history['val_accuracy']):.4f}")
print(f"   Final Training Loss: {vgg16_history.history['loss'][-1]:.4f}")
print(f"   Final Validation Loss: {vgg16_history.history['val_loss'][-1]:.4f}")
print(f"   Total Epochs: {len(vgg16_history.history['accuracy'])}")

print("✅ Training plots generated and saved!")

## 10. Model Evaluation and Metrics

Evaluate the VGG16 model on the test set and generate comprehensive metrics:

In [ ]:
# Load the best model weights
best_model_path = os.path.join(RESULTS_DIR, "VGG16", "VGG16_best.h5")
if os.path.exists(best_model_path):
    vgg16_model.load_weights(best_model_path)
    print("✅ Loaded best model weights")
else:
    print("⚠️ Best model weights not found, using current weights")

# Evaluate VGG16 model on test set
print(f"\n📊 Evaluating VGG16 model on test set...")

# Get predictions
y_true = []
y_pred = []
y_pred_proba = []

print("🔄 Generating predictions...")
for images, labels in test_ds:
    predictions = vgg16_model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred_proba.extend(predictions.flatten())
    y_pred.extend((predictions > 0.5).astype(int).flatten())

# Calculate comprehensive metrics
test_accuracy = accuracy_score(y_true, y_pred)
test_precision = precision_score(y_true, y_pred)
test_recall = recall_score(y_true, y_pred)
test_f1 = f1_score(y_true, y_pred)

# Get training metrics
train_acc = max(vgg16_history.history['accuracy'])
val_acc = max(vgg16_history.history['val_accuracy'])
train_loss = min(vgg16_history.history['loss'])
val_loss = min(vgg16_history.history['val_loss'])

# Create comprehensive metrics dictionary
vgg16_metrics = {
    'model_name': 'VGG16_Brain_Tumor_Classifier',
    'model_type': 'Transfer Learning (Fine-tuned)',
    'architecture': 'VGG16 + Custom Classifier',
    'train_accuracy': float(train_acc),
    'val_accuracy': float(val_acc),
    'test_accuracy': float(test_accuracy),
    'train_loss': float(train_loss),
    'val_loss': float(val_loss),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_f1_score': float(test_f1),
    'epochs_trained': len(vgg16_history.history['accuracy']),
    'total_parameters': int(vgg16_model.count_params()),
    'trainable_parameters': int(sum([tf.keras.backend.count_params(w) for w in vgg16_model.trainable_weights]))
}

# Display results
print(f"\n🎯 VGG16 Model Performance Results:")
print("="*60)
print(f"📈 Training Metrics:")
print(f"   Best Training Accuracy: {train_acc:.4f}")
print(f"   Best Validation Accuracy: {val_acc:.4f}")
print(f"   Best Training Loss: {train_loss:.4f}")
print(f"   Best Validation Loss: {val_loss:.4f}")

print(f"\n🎯 Test Set Performance:")
print(f"   Test Accuracy: {test_accuracy:.4f}")
print(f"   Test Precision: {test_precision:.4f}")
print(f"   Test Recall: {test_recall:.4f}")
print(f"   Test F1-Score: {test_f1:.4f}")

print(f"\n🏗️ Model Configuration:")
print(f"   Total Parameters: {vgg16_metrics['total_parameters']:,}")
print(f"   Trainable Parameters: {vgg16_metrics['trainable_parameters']:,}")
print(f"   Epochs Trained: {vgg16_metrics['epochs_trained']}")

# Save individual model metrics
with open(os.path.join(RESULTS_DIR, "VGG16_metrics.json"), "w") as f:
    json.dump(vgg16_metrics, f, indent=4)

print(f"\n✅ Model evaluation completed!")
print(f"📁 Metrics saved to: VGG16_metrics.json")

## 11. Create Confusion Matrix

Generate confusion matrix and classification report for the VGG16 model:

In [ ]:
# Create confusion matrix for VGG16 model
plt.figure(figsize=(10, 8))

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Create confusion matrix display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap='Blues', values_format='d')

plt.title('VGG16 Brain Tumor Classification - Confusion Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "confusion_matrix.png"), dpi=300, bbox_inches='tight')
plt.show()

# Generate detailed classification report
classification_report_text = classification_report(y_true, y_pred, target_names=class_names)
print(f"\n📋 VGG16 Classification Report:")
print("="*70)
print(classification_report_text)

# Save classification report
with open(os.path.join(RESULTS_DIR, "classification_report.txt"), "w") as f:
    f.write("VGG16 Brain Tumor Classification - Detailed Report\n")
    f.write("="*70 + "\n\n")
    f.write("Model: VGG16 Transfer Learning (Fine-tuned)\n")
    f.write(f"Test Accuracy: {test_accuracy:.4f}\n")
    f.write(f"Test F1-Score: {test_f1:.4f}\n\n")
    f.write("Classification Report:\n")
    f.write("-"*50 + "\n")
    f.write(classification_report_text)
    f.write("\n\nConfusion Matrix:\n")
    f.write("-"*20 + "\n")
    f.write(f"True Negatives (no): {cm[0,0]}\n")
    f.write(f"False Positives (no→yes): {cm[0,1]}\n")
    f.write(f"False Negatives (yes→no): {cm[1,0]}\n")
    f.write(f"True Positives (yes): {cm[1,1]}\n")

# Create a detailed metrics visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('VGG16 Model Performance Analysis', fontsize=16, fontweight='bold')

# Metrics bar chart
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_values = [test_accuracy, test_precision, test_recall, test_f1]
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold']

bars = ax1.bar(metrics_names, metrics_values, color=colors, alpha=0.8)
ax1.set_title('Test Set Metrics', fontweight='bold')
ax1.set_ylabel('Score')
ax1.set_ylim(0, 1)
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# Training vs Validation comparison
train_val_metrics = ['Training Acc', 'Validation Acc', 'Training Loss', 'Validation Loss']
train_values = [train_acc, val_acc, train_loss, val_loss]
x_pos = np.arange(len(train_val_metrics[:2]))

ax2.bar(x_pos, [train_acc, val_acc], color=['blue', 'red'], alpha=0.7)
ax2.set_title('Training vs Validation Accuracy', fontweight='bold')
ax2.set_ylabel('Accuracy')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['Training', 'Validation'])
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3)

# Add value labels
for i, v in enumerate([train_acc, val_acc]):
    ax2.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

# Prediction distribution
ax3.hist(y_pred_proba, bins=30, alpha=0.7, color='purple', edgecolor='black')
ax3.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Decision Threshold')
ax3.set_title('Prediction Probability Distribution', fontweight='bold')
ax3.set_xlabel('Predicted Probability')
ax3.set_ylabel('Frequency')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Confusion matrix as heatmap
im = ax4.imshow(cm, interpolation='nearest', cmap='Blues')
ax4.set_title('Confusion Matrix Heatmap', fontweight='bold')
tick_marks = np.arange(len(class_names))
ax4.set_xticks(tick_marks)
ax4.set_yticks(tick_marks)
ax4.set_xticklabels(class_names)
ax4.set_yticklabels(class_names)
ax4.set_ylabel('True Label')
ax4.set_xlabel('Predicted Label')

# Add text annotations
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    ax4.text(j, i, format(cm[i, j], 'd'),
             ha="center", va="center",
             color="white" if cm[i, j] > thresh else "black",
             fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "performance_analysis.png"), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Confusion matrix and performance analysis generated!")

## 12. Generate Grad-CAM Visualizations

Create Grad-CAM visualizations to understand what the VGG16 model focuses on:

In [ ]:
# Generate Grad-CAM visualizations for the VGG16 model
print(f"🔍 Generating Grad-CAM visualizations for VGG16 model...")

# Create gradcam directory
gradcam_dir = os.path.join(RESULTS_DIR, "gradcam")
os.makedirs(gradcam_dir, exist_ok=True)

# Generate Grad-CAM for VGG16 model
try:
    generate_gradcam(
        model=vgg16_model, 
        dataset=test_ds, 
        save_dir=gradcam_dir, 
        class_names=class_names,
        num_images=5,  # Generate for 5 images
        max_samples=3   # Limit to first 3 batches for speed
    )
    print("✅ Grad-CAM visualizations generated successfully!")
except Exception as e:
    print(f"⚠️  Grad-CAM generation failed: {str(e)}")
    print("Continuing without Grad-CAM...")

# Create a comprehensive sample predictions visualization
plt.figure(figsize=(20, 15))
sample_count = 0
max_samples = 16

print("📸 Creating sample predictions visualization...")

for images, labels in test_ds.take(3):  # Take 3 batches
    predictions = vgg16_model.predict(images, verbose=0)
    
    for i in range(min(len(images), max_samples - sample_count)):
        if sample_count >= max_samples:
            break
            
        plt.subplot(4, 4, sample_count + 1)
        plt.imshow(images[i].numpy())
        
        true_label = class_names[int(labels[i])]
        pred_prob = predictions[i][0]
        pred_label = class_names[1] if pred_prob > 0.5 else class_names[0]
        confidence = pred_prob if pred_prob > 0.5 else (1 - pred_prob)
        
        # Color based on correctness
        color = 'green' if true_label == pred_label else 'red'
        
        plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.3f}', 
                 color=color, fontsize=12, fontweight='bold')
        plt.axis('off')
        sample_count += 1
    
    if sample_count >= max_samples:
        break

plt.suptitle('VGG16 Model - Sample Predictions on Test Set', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "sample_predictions.png"), dpi=300, bbox_inches='tight')
plt.show()

# Create prediction confidence analysis
correct_predictions = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true == pred]
incorrect_predictions = [i for i, (true, pred) in enumerate(zip(y_true, y_pred)) if true != pred]

correct_confidences = [y_pred_proba[i] if y_pred[i] == 1 else 1-y_pred_proba[i] for i in correct_predictions]
incorrect_confidences = [y_pred_proba[i] if y_pred[i] == 1 else 1-y_pred_proba[i] for i in incorrect_predictions]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(correct_confidences, bins=20, alpha=0.7, color='green', label=f'Correct ({len(correct_predictions)})')
plt.hist(incorrect_confidences, bins=20, alpha=0.7, color='red', label=f'Incorrect ({len(incorrect_predictions)})')
plt.xlabel('Prediction Confidence')
plt.ylabel('Frequency')
plt.title('Prediction Confidence Distribution', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
accuracy_by_confidence = []
confidence_bins = np.arange(0.5, 1.01, 0.05)

for i in range(len(confidence_bins)-1):
    lower, upper = confidence_bins[i], confidence_bins[i+1]
    indices = [j for j, conf in enumerate([y_pred_proba[k] if y_pred[k] == 1 else 1-y_pred_proba[k] 
                                          for k in range(len(y_pred))]) 
              if lower <= conf < upper]
    
    if indices:
        bin_accuracy = sum(y_true[j] == y_pred[j] for j in indices) / len(indices)
        accuracy_by_confidence.append(bin_accuracy)
    else:
        accuracy_by_confidence.append(0)

plt.plot(confidence_bins[:-1], accuracy_by_confidence, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Confidence Bin')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Prediction Confidence', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1.1)

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "confidence_analysis.png"), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Sample predictions and confidence analysis completed!")

# Summary statistics
print(f"\n📊 Prediction Analysis Summary:")
print(f"   Total test samples: {len(y_true)}")
print(f"   Correct predictions: {len(correct_predictions)} ({len(correct_predictions)/len(y_true)*100:.1f}%)")
print(f"   Incorrect predictions: {len(incorrect_predictions)} ({len(incorrect_predictions)/len(y_true)*100:.1f}%)")
print(f"   Average confidence (correct): {np.mean(correct_confidences):.3f}")
print(f"   Average confidence (incorrect): {np.mean(incorrect_confidences):.3f}")
print(f"   High confidence correct (>0.9): {sum(1 for c in correct_confidences if c > 0.9)}")
print(f"   High confidence incorrect (>0.9): {sum(1 for c in incorrect_confidences if c > 0.9)}")

## 13. Save Model and Final Results

Save the VGG16 model and generate all required output files:

In [ ]:
# Save the VGG16 model as best_model.h5
best_model_path = os.path.join(RESULTS_DIR, "best_model.h5")
vgg16_model.save(best_model_path)
print(f"✅ VGG16 model saved as: {best_model_path}")

# Save comprehensive metrics for easy comparison with your friend's models
final_metrics = {
    "model_name": "VGG16_Brain_Tumor_Classifier",
    "model_type": "Transfer Learning (Fine-tuned VGG16)",
    "architecture_details": {
        "base_model": "VGG16 (ImageNet pre-trained)",
        "fine_tuning": "Last 4 layers unfrozen",
        "classifier": "GAP + Dense(512) + BN + Dropout(0.5) + Dense(256) + Dropout(0.3) + Dense(1)",
        "input_shape": INPUT_SHAPE,
        "total_parameters": vgg16_metrics['total_parameters'],
        "trainable_parameters": vgg16_metrics['trainable_parameters']
    },
    "training_config": {
        "batch_size": BATCH_SIZE,
        "epochs_trained": vgg16_metrics['epochs_trained'],
        "initial_learning_rate": LEARNING_RATE/10,  # Fine-tuning LR
        "optimizer": "Adam",
        "loss_function": "binary_crossentropy",
        "callbacks": ["ModelCheckpoint", "EarlyStopping", "ReduceLROnPlateau"]
    },
    "performance_metrics": {
        "train_accuracy": vgg16_metrics['train_accuracy'],
        "val_accuracy": vgg16_metrics['val_accuracy'],
        "test_accuracy": vgg16_metrics['test_accuracy'],
        "test_precision": vgg16_metrics['test_precision'],
        "test_recall": vgg16_metrics['test_recall'],
        "test_f1_score": vgg16_metrics['test_f1_score'],
        "train_loss": vgg16_metrics['train_loss'],
        "val_loss": vgg16_metrics['val_loss']
    },
    "dataset_info": {
        "class_names": class_names,
        "train_samples": train_samples,
        "val_samples": val_samples,
        "test_samples": test_samples,
        "total_samples": train_samples + val_samples + test_samples,
        "class_distribution": "Binary classification: yes/no for brain tumor presence"
    },
    "comparison_ready": {
        "accuracy": vgg16_metrics['test_accuracy'],
        "f1_score": vgg16_metrics['test_f1_score'],
        "precision": vgg16_metrics['test_precision'],
        "recall": vgg16_metrics['test_recall'],
        "model_size_mb": os.path.getsize(best_model_path) / (1024*1024) if os.path.exists(best_model_path) else 0
    }
}

# Save the comprehensive metrics
with open(os.path.join(RESULTS_DIR, "metrics.json"), "w") as f:
    json.dump(final_metrics, f, indent=4)

print(f"✅ Comprehensive metrics saved: {os.path.join(RESULTS_DIR, 'metrics.json')}")

# Generate final summary report for easy comparison
summary_report = f"""# VGG16 Brain Tumor Classification - Final Results

## 🎯 Model Performance Summary
- **Model**: VGG16 Transfer Learning (Fine-tuned)
- **Test Accuracy**: {vgg16_metrics['test_accuracy']:.4f} ({vgg16_metrics['test_accuracy']*100:.2f}%)
- **Test Precision**: {vgg16_metrics['test_precision']:.4f}
- **Test Recall**: {vgg16_metrics['test_recall']:.4f}
- **Test F1-Score**: {vgg16_metrics['test_f1_score']:.4f}

## 📊 Comparison Metrics (for your friend's comparison)
```
Accuracy:  {vgg16_metrics['test_accuracy']:.4f}
F1-Score:  {vgg16_metrics['test_f1_score']:.4f}
Precision: {vgg16_metrics['test_precision']:.4f}
Recall:    {vgg16_metrics['test_recall']:.4f}
```

## 🏗️ Model Architecture
- **Base Model**: VGG16 (ImageNet pre-trained)
- **Fine-tuning**: Last 4 layers unfrozen
- **Classifier**: Custom head with BatchNorm and Dropout
- **Parameters**: {vgg16_metrics['total_parameters']:,} total ({vgg16_metrics['trainable_parameters']:,} trainable)

## 🎮 Training Configuration
- **Epochs Trained**: {vgg16_metrics['epochs_trained']}
- **Batch Size**: {BATCH_SIZE}
- **Learning Rate**: {LEARNING_RATE/10} (fine-tuning)
- **Optimizer**: Adam
- **Callbacks**: EarlyStopping, ReduceLROnPlateau

## 📁 Generated Files
- `best_model.h5` - Trained VGG16 model
- `training_plot.png` - Training history visualization
- `confusion_matrix.png` - Model performance matrix
- `metrics.json` - Comprehensive metrics data
- `gradcam/` - Grad-CAM visualizations
- `classification_report.txt` - Detailed performance report

## 📈 Dataset Information
- **Classes**: {class_names}
- **Training Samples**: {train_samples:,}
- **Validation Samples**: {val_samples:,}
- **Test Samples**: {test_samples:,}
- **Total Samples**: {train_samples + val_samples + test_samples:,}

---
*Generated for comparison with other deep learning models*
"""

with open(os.path.join(RESULTS_DIR, "summary_report.md"), "w") as f:
    f.write(summary_report)

print("✅ Summary report generated!")

# List all generated files for verification
print(f"\n📁 Generated files in {RESULTS_DIR}:")
for item in sorted(os.listdir(RESULTS_DIR)):
    item_path = os.path.join(RESULTS_DIR, item)
    if os.path.isfile(item_path):
        size_mb = os.path.getsize(item_path) / (1024*1024)
        print(f"   📄 {item} ({size_mb:.1f} MB)")
    elif os.path.isdir(item_path):
        print(f"   📁 {item}/")
        for subfile in sorted(os.listdir(item_path)):
            subfile_path = os.path.join(item_path, subfile)
            if os.path.isfile(subfile_path):
                size_mb = os.path.getsize(subfile_path) / (1024*1024)
                print(f"      📄 {subfile} ({size_mb:.1f} MB)")

# Final performance summary
print(f"\n🎉 VGG16 Brain Tumor Classification completed successfully!")
print(f"="*70)
print(f"🏆 FINAL RESULTS:")
print(f"   Model: VGG16 Transfer Learning")
print(f"   Test Accuracy: {vgg16_metrics['test_accuracy']:.4f} ({vgg16_metrics['test_accuracy']*100:.2f}%)")
print(f"   Test F1-Score: {vgg16_metrics['test_f1_score']:.4f}")
print(f"   Model Size: {final_metrics['comparison_ready']['model_size_mb']:.1f} MB")
print(f"   Training Time: {vgg16_metrics['epochs_trained']} epochs")
print(f"="*70)
print(f"📁 All results saved in: {RESULTS_DIR}")
print(f"📊 Ready for comparison with your friend's models!")

# Create a simple comparison table for easy reference
comparison_table = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'VGG16_Score': [
        f"{vgg16_metrics['test_accuracy']:.4f}",
        f"{vgg16_metrics['test_precision']:.4f}",
        f"{vgg16_metrics['test_recall']:.4f}",
        f"{vgg16_metrics['test_f1_score']:.4f}"
    ]
})

print(f"\n? Quick Comparison Table:")
print(comparison_table.to_string(index=False))

# Save comparison table
comparison_table.to_csv(os.path.join(RESULTS_DIR, "comparison_metrics.csv"), index=False)
print(f"\n✅ Comparison table saved as: comparison_metrics.csv")